In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd
import tifffile as tiff

from fimpylab import LightsheetExperiment

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.utils import zscore
from lotr.pca import pca_and_phase, get_fictive_heading, fictive_heading_and_fit, \
        fit_phase_neurons,qap_sorting_and_phase
from circle_fit import hyper_fit
from lotr.experiment_class import LotrExperiment
import json

from lotr.plotting.color_utils import get_n_colors

import statsmodels.api as sm

import scipy

In [3]:
from lotr.utils import pearson_regressors

In [101]:
master = Path(r"\\FUNES\Shared\experiments\E0071_lotr\full_ring")
files = list(master.glob("*/*f[0-9]_*"))

master = Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v26_\huc")
#master = Path(r"\\FUNES\Shared\experiments\E0040_motions_cardinal\v21\ls\h2b_wb")

files = list(master.glob("*_f*"))

path = files[2]
path_save = path #/ 'autocorrelation'
path

WindowsPath('//funes/Shared/experiments/E0040_motions_cardinal/v26_/huc/220504_f2')

In [102]:
traces = fl.load(path / "filtered_traces.h5", "/detr")
#coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
#anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")

coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
anat = fl.load(path / "data_from_suite2p_cells.h5", "/anatomy_stack")

#traces[np.isnan(traces)] = 0

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LotrExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 150
'''
time_slices_dict = {"natmov": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn), 
                        "clol": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn), 
                        "cwccw": (500, exp.n_pts // 2),
                        "2dvr": (t_start_s*exp.fn, 2000*exp.fn),
                        "2d": (t_start_s*exp.fn, 2000*exp.fn),
                        "spont": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn),
                        "gainmod": (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn)}
'''
t_lims = (150, exp.n_pts)#[-1]
t_slice = slice(*t_lims)
t_slice
traces_spont = traces[0:t_lims[1], :]

In [103]:
try:
    with open(next(path.glob("*metadata.json"))) as i:
        metadata = json.load(i)
    fish_id = metadata['general']['fish_id']
except:
    fish_id = ""
print(fish_id)

220504_f2


In [104]:
np.shape(traces)

(7200, 16985)

In [105]:
fictive_trajectory = get_fictive_heading(np.shape(traces)[0], df, min_bias=0.1)
phase_traj = np.mod(fictive_trajectory, np.pi * 2) * (180 / np.pi)

#fictive_trajectory = fictive_trajectory[0:t_lims[1]]
#phase_traj = phase_traj[0:t_lims[1]]
#traces = traces_spont
np.shape(traces_spont)

(7200, 16985)

In [106]:
num_bins = 4
bin_size = 360 / num_bins
binned_traj = np.floor(phase_traj / bin_size) * bin_size # 8 directions

In [107]:
# generating regressors
from lotr.default_vals import REGRESSOR_TAU_S, TURN_BIAS
regs = np.zeros((num_bins, np.shape(traces)[0]))

tau_fs = REGRESSOR_TAU_S * fn
kernel = np.exp(-np.arange(1000) / tau_fs)


for i in range(num_bins):
    tmp_val = i * bin_size
    tmp_reg = np.where(binned_traj == tmp_val)[0]
    regs[i, tmp_reg] = 1
    
    regs[i] = np.convolve(regs[i], kernel)[:np.shape(traces)[0]]

In [108]:
np.shape(fictive_trajectory)

(7200,)

In [109]:
fig_sum, ax_sum = plt.subplots(3, 1, figsize=(7,8))
color_list = get_n_colors(num_bins) / 255

ax_sum[0].plot(np.arange(len(fictive_trajectory[:])) / 3, -fictive_trajectory)
ax_sum[1].plot(np.arange(len(fictive_trajectory[:])) / 3, phase_traj)
ax_sum[1].plot(np.arange(len(fictive_trajectory[:])) / 3, binned_traj)

for i in range(num_bins):
    ax_sum[2].plot(np.arange(len(fictive_trajectory[:])) / 3, regs[i] + (i*50), c=color_list[i])
    
    
title_list = ['Heading', 'Binned wrapped heading', 'Heading regressors']
for i in range(3):
    ax_sum[i].axis('off')
    ax_sum[i].set_title(title_list[i])
    
plt.subplots_adjust(hspace=0.5)
file_name = 'spont_heading_regressors_' + str(num_bins) + '_' + fish_id + '.jpg'
fig_sum.savefig(path_save / file_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
if num_bins == 8:
    fig_size = (15,7)
else:
    fig_size = (5,7)

In [111]:
# getting regression values for all traces
reg_mat = pearson_regressors(traces, regs.T).T

In [112]:

nm_row = 2
nm_col = num_bins // nm_row
fig_anatomy2, ax_anatomy2 = plt.subplots(nm_row, nm_col, figsize=fig_size)


for i in range(num_bins):
    r = i // nm_col
    c = np.mod(i, nm_col)
    print(r, c)
    vmax = np.nanmax(reg_mat[:,i])
    ax_anatomy2[r, c].scatter(coords[:, 1], coords[:, 2], c=reg_mat[:,i], cmap='coolwarm', s=3, vmin=-vmax, vmax=vmax)
    ax_anatomy2[r, c].axis('off')
sbar_len = np.round(np.shape(anat)[2] / 8) / 100
file_name = 'heading_regressors_rois_' + str(num_bins) + '_' + fish_id + '.jpg'
fig_anatomy2.savefig(path_save / file_name)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 0
0 1
1 0
1 1


In [113]:
nm_row = 2
nm_col = num_bins // nm_row
fig_anatomy, ax_anatomy = plt.subplots(nm_row, nm_col, figsize=fig_size)


try:
    for i in range(num_bins):
        r = i // nm_col
        c = np.mod(i, nm_col)
        print(r, c)

        labels = reg_mat[:,i]
        labels_fix = np.copy(reg_mat)
        new_ind = np.where((labels > 0.1) | (labels < -0.1))
        labels_fix = labels[new_ind]
        coords_fix = coords[new_ind]

        vmax = np.nanmax(labels_fix)
        ax_anatomy[r, c].scatter(coords_fix[:, 1], coords_fix[:, 2], c=labels_fix, cmap='coolwarm', s=3, vmin=-vmax, vmax=vmax)
        ax_anatomy[r, c].axis('off')
        
except:
    pass
    
sbar_len = np.round(np.shape(anat)[2] / 8) / 100
file_name = 'heading_regressors_rois_filt_' + str(num_bins) + '_' + fish_id + '.jpg'
fig_anatomy.savefig(path_save / file_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 0
0 1
1 0
1 1


In [114]:
# filter traces with negative correlation threshhold
corrmat = np.corrcoef(traces.T)
ng_corr_ind = np.unique(np.where(corrmat < -0.5)[0])
corrmat_neg = corrmat[ng_corr_ind]
traces_selected = traces[:, ng_corr_ind]
np.shape(traces_selected)

(7200, 2431)

In [115]:
nm_row = 2
nm_col = num_bins // nm_row
fig_anatomy3, ax_anatomy3 = plt.subplots(nm_row, nm_col, figsize=fig_size)


for i in range(num_bins):
    r = i // nm_col
    c = np.mod(i, nm_col)
    print(r, c)
    
    labels = reg_mat[:,i]
    labels_fix = np.copy(reg_mat)
    
    labels_fix = labels[ng_corr_ind]
    coords_fix = coords[ng_corr_ind]
    vmax = np.nanmax(labels_fix)
    print(vmax)
    ax_anatomy3[r, c].scatter(coords_fix[:, 1], coords_fix[:, 2], c=labels_fix, cmap='coolwarm', s=3, vmin=-vmax, vmax=vmax)
    ax_anatomy3[r, c].axis('off')
sbar_len = np.round(np.shape(anat)[2] / 8) / 100
file_name = 'spont_neg_corr_rois_filt_' + str(num_bins) + '_' + fish_id + '.jpg'
fig_anatomy3.savefig(path / file_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 0
0.5224888397706672
0 1
0.4235590297081664
1 0
0.4829447540673524
1 1
0.44905387263116975


# np.shape(traces)

In [116]:
############### autocorrelation time constant

In [117]:
def monoExp(x, m, t, b):
    return m * np.exp(-t * x) + b

In [118]:
try:
    with open(path / "*metadata.json") as f:
        metadata = json.load(f)

    fs = int(metadata["imaging"]["microscope_config"]["lightsheet"]["scanning"]["z"]["frequency"]) # sampling frequency
except:
    fs = 3

In [119]:
num_traces = np.shape(traces)[1]
n_lags = 1000 # np.shape(traces)[0]
M_all = traces
print(np.shape(traces))
AC_all = np.zeros((n_lags+1, num_traces))
for i in range(num_traces):
    ys =  sm.tsa.acf(M_all[:,i], nlags=n_lags)
    AC_all[:, i] = ys

(7200, 16985)


C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\statsmodels\tsa\stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,


In [120]:
ac=sm.tsa.acf(traces[:, 300], nlags = n_lags)
ac_len = len(ac)
fig_ac, ax_ac = plt.subplots(2, 2, figsize=fig_size, gridspec_kw={'width_ratios': [3, 1]})

for i in range(100):
    ax_ac[0, 0].plot(AC_all[:, i])
    ax_ac[0, 1].plot(AC_all[:200, i])

ax_ac[1, 0].imshow(AC_all.T, cmap = 'coolwarm', aspect='auto')      
ax_ac[1, 1].imshow(AC_all.T[:, :200], cmap = 'coolwarm', aspect='auto')      

for i in range(2):
    ax_ac[0, i].axis('off')
    ax_ac[1, i].axis('off')
    
file_name = 'ac_tau_traces_' + fish_id + '.jpg'
fig_ac.savefig(path_save / file_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:

All_ac = np.empty((len(ac),M_all.shape[1]))
All_ac[:] = np.nan

ac_len = 101
xs = (1 / fs)*np.arange(ac_len)

All_Tau =  np.empty((M_all.shape[1]))
All_Tau[:] = np.nan

All_mag =  np.empty((M_all.shape[1]))
All_mag[:] = np.nan

All_b =  np.empty((M_all.shape[1]))
All_b[:] = np.nan


error_count=0
p0 = (1, .1, 0) # start with values near those we expect
for i in range(M_all.shape[1]):
    ys =  sm.tsa.acf(M_all[:,i], nlags=100)
    try:
        params, cv = scipy.optimize.curve_fit(monoExp, xs, ys, p0)
        m, t, b = params    
        All_mag[i] = m
        All_b[i] = b
        tauSec = (1 / t) / fs
        All_Tau[i] = t
    except:
        All_Tau[i] = None
        error_count += 1
print(error_count)

C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\statsmodels\tsa\stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,


26


In [122]:
fig_anatomy4, ax_anatomy4 = plt.subplots(1, 4, figsize=(8, 2))
lags = np.asarray([5, 10, 20, 50]) * fs

for i in range(4):
    ax_anatomy4[i].scatter(coords[:, 1], coords[:, 2], c=AC_all[lags[i], :], cmap='Greens', s=3, vmin=0, vmax=1)
    ax_anatomy4[i].axis('off')
    ax_anatomy4[i].set_title('lag ' + str(lags[i] // fs) + ' sec')

file_name = 'ac_rois_corr_at_lag_' + fish_id + '.jpg'
fig_anatomy4.savefig(path_save / file_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
fig_anatomy5, ax_anatomy5 = plt.subplots(1, 1, figsize=(3,3))
ax_anatomy5.scatter(coords[:, 1], coords[:, 2], c=All_Tau, cmap='hot', s=3, vmin=0, vmax=0.4)
ax_anatomy5.axis('off')

file_name = 'ac_rois_tau_' + fish_id + '.jpg'
fig_anatomy5.savefig(path_save / file_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …